In [1]:
from dataset import YouCookII
from dataset import YouCookIICollate
from torch.utils.data import DataLoader
from loss import *
from accuracy import *
from transformers import get_linear_schedule_with_warmup
from model import Model

import numpy as np
import torch
import matplotlib.pyplot as plt

def train(model, num_actions, batch_size, epochs=25, lr=0.001, MAX_DETECTIONS=20):
    dataset = YouCookII(num_actions, "/h/sagar/ece496-capstone/datasets/ycii")
    train_size = int(len(dataset) * (2/3))
    valid_size = int(len(dataset) - train_size)
    
    train_dataset, valid_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size])
    
    collate = YouCookIICollate(MAX_DETECTIONS=MAX_DETECTIONS)
    
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate, drop_last=True)
    valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate, drop_last=True)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(optimizer, int(0.2*epochs), epochs)

    train_loss = np.zeros(epochs)
    valid_loss = np.zeros(epochs)
    
    train_accuracy = np.zeros(epochs)
    valid_accuracy = np.zeros(epochs)
    
    for epoch in range(epochs):
        model.train()
        
        epoch_loss = 0.
        num_batches = 0
        
        total = 0
        correct = 0
        
        for data in train_dataloader:
            _, bboxes, features, steps, entities, entity_count, _, _ = data
            
            # Zero out any gradients.
            optimizer.zero_grad()
            
            # Run inference (forward pass).
            loss_data, VG, RR = model(batch_size, num_actions, steps, features, bboxes, entities, entity_count)
            
            # Loss from alignment.
            loss_ = compute_loss_batched(loss_data)

            # Backpropagation (backward pass).
            loss_.backward()

            # Update parameters.
            optimizer.step()
            
            epoch_loss += loss_
            num_batches += 1
        
        # Scheduler update.
        scheduler.step()
        epoch_loss = epoch_loss / num_batches
        
        # Save loss and accuracy at each epoch and plot.
        train_loss[epoch] = float(epoch_loss)
        train_accuracy[epoch] = get_alignment_accuracy(model, train_dataloader, batch_size, num_actions) 
        
        valid_loss[epoch] = get_alignment_loss(model, valid_dataloader, batch_size, num_actions) / num_batches
        valid_accuracy[epoch] = get_alignment_accuracy(model, valid_dataloader, batch_size, num_actions)

        print("Epoch {} - Train Loss: {}, Validation Loss: {}".format(epoch + 1, train_loss[epoch], valid_loss[epoch]))
    
    plt.figure()
    plt.plot(train_loss, label='train loss')
    plt.plot(valid_loss, label='valid loss')
    plt.legend()
    
    plt.figure()
    plt.plot(train_accuracy, label='train accuracy')
    plt.plot(valid_accuracy, label='valid accuracy')
    plt.legend()
    
    plt.show()
        
    return train_loss, valid_loss, train_accuracy, valid_accuracy

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Model(device)

In [ ]:
# Trainer.

train_loss, valid_loss, train_accuracy, valid_accuracy = train(model, 8, 4, epochs=25, lr=1e-3)

In [ ]:
# Evaluation.

from eval_fi import eval_all_dataset
eval_all_dataset(model)

--------------------------------------------------
Action 1: in a bowl, add ground beef, salt, pepper, 1 egg, dijon mustard and bread crumbs
Best IoU possible = 0.8062977194786072
Random Candidate IoU = 0.17177285254001617
Chosen Frame IoU: 0.6060513257980347
Best IoU possible = 0.617510199546814
Random Candidate IoU = 0.3304663300514221
Chosen Frame IoU: 0.4029606282711029
This entity has no ground truth bounding box
This entity has no ground truth bounding box
This entity has no ground truth bounding box
This entity has no ground truth bounding box
This entity has no ground truth bounding box
--------------------------------------------------
Action 2: mix the ingredients with hand
Best IoU possible = 0.6298992037773132
Random Candidate IoU = 0.3535516560077667
Chosen Frame IoU: 0.38315606117248535
This entity has no ground truth bounding box
--------------------------------------------------
Action 3: Add vegetable oil in a pan
Best IoU possible = 0.7462273240089417
Random Candidate

--------------------------------------------------
Action 1: cut the cabbage into small pieces
Best IoU possible = 0.9013723731040955
Random Candidate IoU = 0.0
Chosen Frame IoU: 0.0
This entity has no ground truth bounding box
--------------------------------------------------
Action 2: Add carawat seeds, sea salt
Best IoU possible = 0.11877390742301941
Random Candidate IoU = 0.04024482518186186
Chosen Frame IoU: 0.08038705587387085
This entity has no ground truth bounding box
--------------------------------------------------
Action 3: mix them evenly in the cabbage
Best IoU possible = 0.22755128145217896
Random Candidate IoU = 0.0
Chosen Frame IoU: 0.050209250301122665
Best IoU possible = 0.8768255710601807
Random Candidate IoU = 0.08650709688663483
Chosen Frame IoU: 0.7476661801338196
--------------------------------------------------
Action 4: Add some leftover brine in a jar
Best IoU possible = 0.23314441740512848
Random Candidate IoU = 0.07153001427650452
Chosen Frame IoU: 0.042

Best IoU possible = 0.6718937158584595
Random Candidate IoU = 0.30473798513412476
Chosen Frame IoU: 0.4231456052249209
Best IoU possible = 0.7325146198272705
Random Candidate IoU = 0.04205983504652977
Chosen Frame IoU: 0.6572661399841309
--------------------------------------------------
Action 9: Pour the marinade over the ribs
Best IoU possible = 0.4205766022205353
Random Candidate IoU = 0.11928954720497131
Chosen Frame IoU: 0.2673911154270172
Best IoU possible = 0.5321287512779236
Random Candidate IoU = 0.0
Chosen Frame IoU: 0.3496845066547394
Mean Upper Bound IoU: 0.47700080275535583, Mean Random IoU: 0.07666769623756409, Mean Model IoU: 0.20209278166294098
Top-1 acc@0.5:
Proposal Upper Bound: 0.5333333333333333, Random: 0.0, Model: 0.13333333333333333
--------------------------------------------------
Action 1: boil water in a pot with salt and oil
Best IoU possible = 0.6374806761741638
Random Candidate IoU = 0.5610633492469788
Chosen Frame IoU: 0.5688717365264893
Best IoU possibl

In [ ]:
# Visualizer.

from visualizer import inference

YCII = "/h/sagar/ece496-capstone/datasets/ycii"
FI = "/h/sagar/ece496-capstone/datasets/fi"

VG, RR = inference(model, 10, 1, FI)

In [ ]:
# Saving and loading weights.

SAVE = False
LOAD = False

if SAVE:
    torch.save(model.state_dict(), "/h/sagar/ece496-capstone/weights/t1")
    
if LOAD:
    model.load_state_dict(torch.load("/h/sagar/ece496-capstone/weights/t1"))

In [ ]:
# Reload modules.

import importlib
import visualizer
import eval_fi
import model as mdl
import loss

importlib.reload(visualizer)
importlib.reload(eval_fi)
importlib.reload(mdl)
importlib.reload(loss)